In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

import datetime
from datetime import date
import random

/Users/polinab/.virtualenvs/base/lib/python3.8/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
def random_dates(start, end, n=10):

    start_u = start.value//10**9
    end_u = end.value//10**9

    return pd.to_datetime(np.random.randint(start_u, end_u, n), unit='s')

In [14]:
vax_desc = {'pcr':'PCR Test',
               'rec':'Proof Of Recovery',
               'jj':'Johnson & Johnson',
               'az1':'Astra-Zeneca Dose 1',
               'az2':'Astra-Zeneca Dose 2'}

vax_id_type = {'pcr':'test',
               'rec':'rec',
               'jj':'vax',
               'az1':'vax1',
               'az2':'vax2'}
vax_duration = {'test':3,'vax1':0,'vax2':365,'vax':365,'rec':180}
immunization_ids = [key for key,value in vax_id_type.items()]

patient_id_unique = random.sample(range(10000, 99999), 80)
patient_ids = random.choices(patient_id_unique, k=100) # patients can get vaxed multiple times
patient_immunization_ids = ['pcr']*50 +random.choices(immunization_ids, k=50)

cities = {'San Francisco':'USA',
          'New York':'USA',
          'Barcelona':'Spain',
          'Paris':'France',
          'London':'UK',
          'Milan':'Italy'
         }
med_center_id_unique = random.sample(range(1000, 9999), 30)
med_center_cities = random.choices([c for c in cities.keys()], k=30)
med_center_countries = [cities[c] for c in med_center_cities]

# generate random vax dates 
start = pd.to_datetime('2021-01-01')
end = pd.to_datetime(date.today())
immunization_dates = random_dates(start, end,n=100)

med_cen_ids = random.sample(range(10000, 90000), 100)



In [15]:
# Immunization dimension table
vax_dims = {'id':immunization_ids,
            'name':[vax_desc[vid] for vid in immunization_ids],
            'type':[value for key,value in vax_id_type.items()],
            'immune_days':[vax_duration[key] for key in vax_id_type.values()]}

vax_dim_df = pd.DataFrame.from_dict(vax_dims)
vax_dim_df.head()

,id,name,type,immune_days
0,pcr,PCR Test,test,3
1,rec,Proof Of Recovery,rec,180
2,jj,Johnson & Johnson,vax,365
3,az1,Astra-Zeneca Dose 1,vax1,0
4,az2,Astra-Zeneca Dose 2,vax2,365


In [20]:
# fact table

vax_records = {'patient_id':patient_ids,
               'immunization_id': patient_immunization_ids,
               'immunization_date': immunization_dates,
               'medical_center_id': med_cen_ids}

vax_rec_df = pd.DataFrame.from_dict(vax_records)
vax_rec_df['immunization_validity_date']=vax_rec_df.apply(
    lambda x: x['immunization_date']+ datetime.timedelta(days=vax_duration[vax_id_type[x.immunization_id]]),axis=1)
vax_rec_df.head()

,patient_id,immunization_id,immunization_date,medical_center_id,immunization_validity_date
0,96858,pcr,2021-04-20 19:24:28,47827,2021-04-23 19:24:28
1,76257,pcr,2021-01-06 07:24:49,65527,2021-01-09 07:24:49
2,44742,pcr,2021-01-16 18:46:10,66282,2021-01-19 18:46:10
3,23743,pcr,2021-02-15 09:50:40,23736,2021-02-18 09:50:40
4,23040,pcr,2021-01-05 09:56:04,13680,2021-01-08 09:56:04


In [21]:
med_cen_records = {'med_center_id':med_center_id_unique,
                   'name':[f'Dummy Name {i}' for i in med_center_id_unique],
                   'city': med_center_cities,
                   'country':med_center_countries}
med_cen_df = pd.DataFrame.from_dict(med_cen_records)
med_cen_df.head()


,med_center_id,name,city,country
0,5049,Dummy Name 5049,Milan,Italy
1,7909,Dummy Name 7909,London,UK
2,4458,Dummy Name 4458,New York,USA
3,2941,Dummy Name 2941,New York,USA
4,8425,Dummy Name 8425,Barcelona,Spain


In [22]:

from_to_combos = [[i,j] for j in cities for i in cities if i!=j]
from_to = random.choices(from_to_combos, k=50)

'''
for immunity status business logic:
get most latest immunity status of customer
see if the travel date is in range
'''

travel_records = {'customer_id':random.choices(patient_ids, k=50),
                  'travel_date':random_dates(start, end,n=50),
                  'from_city':[e[0] for e in from_to],
                  'to_city':[e[1] for e in from_to]
                 }

travel_df = pd.DataFrame.from_dict(travel_records)


travel_df['immunity_end_date'] = travel_df.apply(
    lambda x: vax_rec_df[vax_rec_df.patient_id == x.customer_id].immunization_validity_date.max(),axis=1)
travel_df['immunity_status'] = travel_df.apply(lambda x: True if x.immunity_end_date > x.travel_date else False, axis=1)
travel_df


,customer_id,travel_date,from_city,to_city,immunity_end_date,immunity_status
0,87875,2021-02-28 07:08:42,London,Milan,2022-04-14 11:38:06,True
1,46729,2021-01-06 04:15:08,Paris,New York,2021-04-13 15:01:55,True
2,31116,2021-05-03 07:59:00,Barcelona,London,2021-05-14 11:05:19,True
3,91388,2021-04-30 15:46:36,Barcelona,San Francisco,2022-05-07 14:31:30,True
4,20285,2021-03-03 17:48:45,San Francisco,Milan,2021-03-04 23:49:37,True
5,84586,2021-04-08 18:04:04,San Francisco,London,2021-09-15 21:16:43,True
6,21315,2021-01-19 15:54:13,Barcelona,Paris,2021-04-07 21:43:59,True
7,21315,2021-04-30 21:58:33,Milan,San Francisco,2021-04-07 21:43:59,False
8,84612,2021-06-03 13:20:48,New York,Milan,2022-05-18 02:47:41,True
9,87875,2021-02-16 15:20:10,San Francisco,Barcelona,2022-04-14 11:38:06,True
